In [3]:
import torch
from   torch import nn

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:

def rdm():
    """
    This is a test function you can call to see if u have imported all from model.ipynb if you 
    called as import 'from model.ipynb import *'. Note that there may be function code errors which may require more
    diagnosing. 
    """
    print("You were able to access a function within `model.ipynb`")

# Steps for the dataloader to either train or test model
* train_epoch() trains the model on that epoch on the dataloader given
* test_epoch() tests the model on that epoch on the dataloader given

In [5]:
# a step for each train epoch
def train_epoch(model:torch.nn.Module,
                dataloader: torch.utils.data.DataLoader,
                optimizer:torch.optim.Optimizer,
                loss_fn: torch.nn.Module,
                device=device):
    
    """
    This is a train step for an epoch during its training process. Within this train step, we will iterate through a model in
    training mode with the training data in batches. 
    """
    
    # put it in train mode
    model.train()
    
    # setup train loss and train accuracy that we will calculate per epoch 
    train_loss, train_acc = 0, 0
    
    # loop through dataloader data batches
    for batch, (X, y) in enumerate(dataloader):
        
        # target device
        X, y = X.to(device), y.to(device)
        
        # 1. forward pass which outputs model logits
        y_pred = model(X)
        
        # 2. loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()
        
        # 3. optimize the zero grad
        optimizer.zero_grad()
        
        # 4. loss backwards
        loss.backward()
        
        # 5. optimizer step
        optimizer.step()
        
        # calculate the accuracy metric for the current batch (y_pred_class is the predicted value)
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class==y).sum().item()/len(y_pred)

        
    # adjust metric accuracy loss and accuracy per batch
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)

    return train_loss, train_acc  

In [6]:
# Test epoch
def test_epoch(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               device=device):
    """
    This is a test step for an epoch during its testing process. Within this train step, we will iterate through a model in
    test (eval) mode with the testing data in batches. 
    """
    # put in evaluation mode (to not modify any parameters of the model)
    model.eval()
    
    # setup the test loss and test accuracy values
    test_loss, test_acc = 0, 0
    
    # turn on inference model for testing
    with torch.inference_mode():
        
        # lop through the dataloader batch
        for batch, (X, y) in enumerate(dataloader):
            
            # send data to the target device
            X, y = X.to(device), y.to(device)
            
            # 1. forward pass
            test_pred_logits = model(X)
            
            # 2. calculate the loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()
            
            # calculate accuracy for X, y batch
            test_pred_label = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_label == y).sum().item()/len(test_pred_label))
            
        # adjust metric to get the average of both test loss and accuracy
        
        test_loss = test_loss / len(dataloader)
        test_acc = test_acc / len(dataloader)
        
    return test_loss, test_acc   

In [7]:
# the train function  that takes in various parameters (needs model, optimizer, loss, dataloader)
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 20,
          device=device,
          display_epoch: bool = True):

    from tqdm.auto import tqdm    

    # empty dictionary for our results
    results = {
        'train_loss': [],
        'train_acc': [],
        'test_loss': [],
        'test_acc': []
    }
    
    # loop through the training and testing epochs for in range(epochs) (default 20)
    
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_epoch(model=model,
                                           dataloader=train_dataloader,
                                           optimizer=optimizer,
                                           loss_fn=loss_fn,
                                           device=device
                                           )
        test_loss, test_acc = test_epoch(model=model,
                                         dataloader=test_dataloader,
                                         loss_fn=loss_fn,
                                         device=device)
        
        # print out information regarding model results per epoch (default set to true)
        if display_epoch:
           print(f"Epoch: {epoch} | Train loss: {train_loss:.4f} | Test loss: {test_loss:.4f} | Test acc: {test_acc:.4f}") 
        
        # update the results within the dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
    
    
    return results
    

In [20]:
class Food_Model(nn.Module):
    """
    A food classifier module that follows the architecture of a CNN + modificaitons that I added.
    """
    def __init__(self, 
                 input_shape: int,
                 hidden_units: int,
                 output_shape: int) -> None:
        """
        input_shape - our case will be 3 when we input due to the shape of x
        hidden_units - amount of intermediate layers between the input and output layer.
        output_shape - amount of classes that we got (will be set to the len of classes)
        """
        super().__init__()
        #
        #
        # will block it up for convenience sake
        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=4,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=4,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,
                        stride=3)  # Note that the default is the same as kernel_size 

        )
        #
        #
        # Block 2
        self.block_2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=4,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=4,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,
                        stride=3)  # Note that the default is the same as kernel_size 

        )
        #
        #
        # classifier
        self.classify = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units * 1 * 1,
                      out_features=output_shape)
        )

    def forward(self, x):
        x = self.block_1(x)
        print(f"shape of x: {x.shape}")
        x = self.block_2(x)
        print(f"shape after 2nd block {x.shape}")
        x = self.classifier(x)
        print(f"after classifier: {x.shape}")
        return x   